In [5]:
import os
loc = '/scratch/koustuvs/compositionality/'

In [14]:
test_labels = []
with open(os.path.join(loc, 'step_0', 'labels.test'),'r') as fp:
    for line in fp:
        test_labels.append(line.rstrip())

In [57]:
train_labels = []
with open(os.path.join(loc, 'step_0', 'labels.train'),'r') as fp:
    for line in fp:
        train_labels.append(line.rstrip())

In [15]:
from collections import Counter

In [16]:
label_ct = Counter(test_labels)

In [56]:
label_ct

Counter({'#': 5741238,
         '>': 735215,
         'v': 718196,
         '<': 728956,
         '|': 731653,
         '=': 9778,
         '^': 21341})

In [58]:
label_tr = Counter(train_labels)

In [59]:
label_tr

Counter({'<': 70878,
         '#': 680875,
         'v': 70065,
         '|': 71445,
         '>': 71180,
         '=': 667,
         '^': 1699})

In [10]:
import pandas as pd
df = pd.read_csv(os.path.join(loc, 'step_0','step_0_BGRUlastEncoder_outp.csv'))

In [11]:
len(df)

8686377

In [12]:
len(test_labels)

8686377

In [17]:
def accuracy(label='#'):
    return len(df[(df['true_target'] == label) & (df['predicted'] == label)]) / len(df[df['true_target'] == label])

In [20]:
for key in label_ct:
    acc = accuracy(key)
    print(key, '-->', acc)

# --> 0.9609976106198698
> --> 0.9122134341655163
v --> 0.9235445477279183
< --> 0.9226949226016385
| --> 0.9220504802139812
= --> 0.9995909183882185
^ --> 0.9190759570779251


In [21]:
df.head()

,Unnamed: 0,s_1,s_2,true_target,predicted
0,0,<s> some rose N11.4 hum V8.2 </s>,<s> all N11.2 sing not V8.2 </s>,#,#
1,1,<s> all N17.4 hum not V14.0 </s>,<s> all rose N17.1 not V14.6 </s>,#,#
2,2,<s> all red N26.1 sing not V22.5 </s>,<s> all red N26.4 sing not V22.1 </s>,#,#
3,3,<s> some N8.2 sing not V5.2 </s>,<s> all rose N8.1 sing not V5.0 </s>,#,#
4,4,<s> some red N51.4 not V47.4 </s>,<s> all red N51.1 sing V47.4 </s>,#,#


In [42]:
import json
meta = json.load(open(os.path.join(loc, 'step_0','meta_data.json')))

In [43]:
len(meta)

1

In [41]:
list(meta['sentences'].keys())[0]

NameError: name 'meta' is not defined

In [1]:
import os
import json
import re

In [2]:
import sys
sys.path.append("../Code/systematicity-tests/")
from sentence import Sentence
from item import Item


In [7]:
def get_item(tup, correct): 
	prem = tup[0].replace(")", '')
	hypo = tup[1].replace(")", '')
	prem = prem.replace("u'", '')
	hypo = hypo.replace("u'", '')
	prem = re.split(r'\(\w+', prem)
	hypo = re.split(r'\(\w+', hypo)
	prem = Sentence(prem[2], prem[3], prem[4], prem[5], prem[6], prem[7], prem[8])
	hypo = Sentence(hypo[2], hypo[3], hypo[4], hypo[5], hypo[6], hypo[7], hypo[8])
	pair = Item(prem, hypo, correct, '' ,1)
	return(pair)

In [8]:
TCN = {}
WUG = {}
IdToStr = {}

a = json.load(open(os.path.join(loc, 'step_0','meta_data.json')))
##I can't figure out why, but a is a dict with one entry (a nested dict, where the keys are actually sentences)
#so we have to un-nest it: 
for key in a: 
    StrToId= a[key]
    break
#then build a dict to handle the reverse pointing: 
for pair in StrToId: 
    IdToStr[StrToId[pair]] = pair


with open(os.path.join(loc, 'step_0','format_out.jsonl')) as fp: 
    for line in fp:
        d = json.loads(line.rstrip())
        atup = IdToStr[d['PairA'][0]], IdToStr[d['PairA'][1]]
        bpairlist = d['PairB']
        acorrect = d['PairAcorrectlabel']
        apredict = d['PairApredictedlabel']
        bcorrect = d['PairBcorrectlabel']
        bpredict = d['PairBPredictedlabel']
        Atype = d['PairAtesttype']
        apair = get_item(atup, acorrect)
        if Atype == 'TCN':
            for each in zip(bpairlist, bcorrect, bpredict):
                btup = IdToStr[each[0][0]], IdToStr[each[0][1]]		
                bpair = get_item(btup, each[1])
                TCN[apair] = bpair.premise, bpair.hypothesis, acorrect, apredict, each[1], each[2], apair.diff(bpair)
        if (Atype == 'TIW') | (Atype == 'TCW'):
            for each in zip(bpairlist, bcorrect, bpredict):
                btup = IdToStr[each[0][0]], IdToStr[each[0][1]]
                bpair = get_item(btup, each[1])
                WUG[apair] = bpair.premise, bpair.hypothesis, acorrect, apredict, each[1], each[2], apair.diff(bpair)

In [9]:
len(TCN)

97427

In [10]:
len(WUG)

48621

In [12]:
list(TCN.values())[0]

(<sentence.Sentence at 0x7ff65325d588>,
 '#',
 '#',
 '<',
 '<',
 (['   ', '   that sing'], 'hypothesis'))

In [26]:
def get_acc(d, label):
    tot = 0
    cor = 0
    for line in d:
        if line['PairAcorrectlabel'] == label:
            tot +=1
            if line['PairApredictedlabel'] == label:
                cor +=1
    return (cor/tot), tot

In [28]:
def print_stats(test_type='TCN'):
    pair_lines_test = [p for p in pair_lines if p['PairAtesttype'] == test_type]
    for key in label_ct:
        acc, tot = get_acc(pair_lines_test, key)
        print(key, '-->', acc, '/', tot)

In [21]:
pair_lines = []
with open(os.path.join(loc, 'step_0','format_out.jsonl')) as fp: 
    for line in fp:
        d = json.loads(line.rstrip())
        pair_lines.append(d)

In [22]:
len(pair_lines)

271987

In [34]:
print_stats('TIW')

# --> 0.7812122167902014 / 19023
> --> 0.5158227848101266 / 1896
v --> 0.5860042735042735 / 1872
< --> 0.5601092896174863 / 1830
| --> 0.5738953783646521 / 1969
= --> 1.0 / 23
^ --> 0.8461538461538461 / 39


In [33]:
print_stats('TIR')

ZeroDivisionError: division by zero

In [24]:
len(pair_lines_tcn)

106661

In [52]:
ep = [p for p in pair_lines_tcn if p['PairAcorrectlabel'] == '^']

In [60]:
for e in ep:
    print(e['PairA'])

[35282, 35283]
[37674, 21619]
[10029, 26151]
[1886, 2192]
[55066, 42204]
[75429, 23793]
[33646, 33644]
[36502, 60886]
[80962, 11198]
[81067, 6255]
[28516, 11606]
[80564, 78984]
[30060, 62352]
[23394, 23396]
[85125, 11198]
[27024, 87142]
[21875, 77079]
[15439, 11895]
[8094, 74710]
[72133, 68330]
[26601, 65767]
[7469, 5115]
[66629, 2525]
[85785, 81242]
[1325, 27958]
[3357, 53354]
[29473, 10647]
[83480, 28652]
[32581, 5962]
[34694, 73422]
[20751, 20753]
[58796, 56572]
[79119, 10251]
[66331, 66330]
[52125, 5398]
[83495, 46057]
[31333, 28621]
[51163, 89532]
[54889, 1855]
[71984, 23417]
[88376, 57471]
[89040, 51972]
[80138, 48194]
[81039, 87986]
[17564, 89523]
[3529, 15825]
[16713, 31359]
[42955, 87770]
[54400, 36854]
[80751, 24537]
[74179, 76811]
[81030, 69089]
[87113, 88034]
[83050, 55368]
[34464, 65991]
[37844, 25161]
[86280, 89881]
[40002, 66535]
[87439, 48571]
[42029, 54201]
[31193, 43245]
[207, 20353]
[12930, 78463]
[69863, 84607]
[39094, 46097]
[42606, 36798]
[4600, 26762]
[84820, 547

In [44]:
id2sent = {v:k for k,v in meta['sentences'].items()}

In [66]:
id2sent[42688]

'(S (Q some)(A red)(N N39.5)(C )(R that hum)(Neg not)(V V35.3))'

In [65]:
id2sent[67042]

'(S (Q all)(A red)(N N39.5)(C )(R that hum)(Neg )(V V35.3))'